<a href="https://www.kaggle.com/code/onyekaonwudinjor/recommender-system-using-knn?scriptVersionId=123177032" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin-1' , sep=',')

In [4]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
df.duplicated().sum()

0

In [7]:
df = df.drop(["UserName", "ScreenName", "Location", "TweetAt"], axis = 1)

# Cleaning the OriginalTweet


In [8]:
df.OriginalTweet[:1]

0    @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
Name: OriginalTweet, dtype: object

In [9]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [10]:
from nltk.corpus import stopwords

In [11]:
stemmer = PorterStemmer()

In [12]:
stop_words= stopwords.words("english")

In [13]:
def clean(OriginalTweet):
    OriginalTweet = re.sub("[^A-Za-z1-9 ]", "", OriginalTweet)
    OriginalTweet = OriginalTweet.lower()
    tokens = word_tokenize(OriginalTweet)
    OriginalTweet_list = []
    for token in tokens:
        if token not in stop_words:
            OriginalTweet_list.append(stemmer.stem(token))
    return " ".join(OriginalTweet_list)

In [14]:
df.OriginalTweet[:4]

0    @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1    advice Talk to your neighbours family to excha...
2    Coronavirus Australia: Woolworths to give elde...
3    My food stock is not the only one which is emp...
Name: OriginalTweet, dtype: object

In [15]:
test_OriginalTweet = df.OriginalTweet[0]

In [16]:
clean(test_OriginalTweet)

'menyrbi philgahan chrisitv httpstcoifz9fan2pa httpstcoxx6ghgfzcc httpstcoi2nlzdxno8'

In [17]:
df.OriginalTweet = df.OriginalTweet.apply(clean)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer()

In [20]:
OriginalTweet_matrix = vectorizer.fit_transform(df.OriginalTweet)

In [21]:
OriginalTweet_matrix = OriginalTweet_matrix.toarray()

In [22]:
from sklearn.neighbors import NearestNeighbors

In [23]:
nn = NearestNeighbors()

In [24]:
nn.fit(OriginalTweet_matrix)

NearestNeighbors()

In [25]:
def inference(OriginalTweet):
    OriginalTweet = clean(OriginalTweet)
    OriginalTweet_matrix = vectorizer.transform([OriginalTweet])
    return nn.kneighbors(n_neighbors=5, X = OriginalTweet_matrix, return_distance=False)

In [26]:
inference(test_OriginalTweet)

array([[    0, 22113, 18818, 12401, 39432]])

In [27]:
df.iloc[[    0, 22113, 18818, 12401, 39432]].OriginalTweet

0        menyrbi philgahan chrisitv httpstcoifz9fan2pa ...
22113    chrisitv asda case fooddrink product need supe...
18818    itvnew chrisitv given petrol station plug tell...
12401    sometim hard describ enjoy visit live germani ...
39432    lukewearechang f offici diagnosi f stay home s...
Name: OriginalTweet, dtype: object

# K Means Recommender

In [28]:
from sklearn.cluster import KMeans